In [1]:
import numpy as np
import pandas as pd
import os

/Applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Read labels

In [2]:
tabledir = "../tables/"

In [3]:
fn = f"{tabledir}/2017-06-mammo_tables/df_dcm_reports_birads_path_indic_dens_birad_wi_year_noreport_nodupl.csv.gz"
df_bt = pd.read_csv(fn, usecols=["id", "BT_case"])
df_bt.set_index("id", inplace=True)
df_bt = ~df_bt.isnull()
df_bt.columns = ["digital"]
df_bt.shape

(772423, 1)

In [4]:
infile = f"{tabledir}/spotmag_predictions/train_test_split-2018-02-16-within7e5-label.csv"
dflab = pd.read_csv(infile, index_col='id')
dflab[:5]

,set,label,view
id,,,
1013372709_1.2.840.113654.2.70.1.175625299786291545159233542096043464711_3_1,test,normal,N
1028995243_1.2.840.113654.2.70.1.56947963181878834591544466761404805157_45576_2,test,normal,N
1105112884_1.2.840.113654.2.70.1.178729598744204462442695104630823323474_8905_2,test,normal,N
1185125156_1.2.840.113654.2.70.1.45840593750642722243371816041014016032_2_4,test,normal,N
1496452586_1.2.840.113654.2.70.1.5582568668770891599992528318631583880_1351_4,test,normal,N


## Read header-based predictions

In [5]:
infile = f"{tabledir}/spotmag_predictions/all_predictions_glmnet.tab"
dfpred_glmnet = pd.read_table(infile, index_col=0)
dfpred_glmnet.columns = [cc.replace("predictions", "score") for cc in dfpred_glmnet.columns]
dfpred_glmnet.shape

(772367, 1)

In [6]:
infile = f"{tabledir}/spotmag_predictions/all_predictions_allmodels_trained_on_train.tab"
dfpred = pd.read_table(infile, index_col=0)
dfpred.columns = [cc.replace("predictions", "score") for cc in dfpred.columns]
dfpred.index.name = 'id'
print(dfpred.shape)
dfpred[:5]

(772367, 5)


,score_gbm,score_xgb,score_rpart,score_xgbt,ViewModifierCodeMeaning
id,,,,,
2454166001_1.2.840.113654.2.70.1.269947926355209368181920716215505958953_149405_2104556,0.009005,0.020207,0.006882,0.059474,NaN
2454166001_1.2.840.113654.2.70.1.269947926355209368181920716215505958953_149405_2104557,0.013337,0.016762,0.006882,0.059660,NaN
2454166001_1.2.840.113654.2.70.1.269947926355209368181920716215505958953_149484_2141538,0.013337,0.016762,0.006882,0.061051,NaN
2454166001_1.2.840.113654.2.70.1.269947926355209368181920716215505958953_149484_2141537,0.013337,0.016762,0.006882,0.061051,NaN
3337971863_1.2.840.113654.2.70.1.337982194343327746313656933304494759333_1_1,0.031560,0.059142,0.006882,0.157488,NaN


In [7]:
if 'set' not in dfpred.columns:
    dfpred = dfpred.merge(dflab,  left_index=True, right_index=True, how='left')
    print(dfpred.shape)

(772367, 8)


In [8]:
colmap = {"N":"normal", "M": "magn/spot",
          "T":"stereotactic", "W":"wire loc", "X":"other"}
view_counts = dfpred[~dfpred.view.isnull()].view.map(lambda x: colmap[x]).value_counts()
view_counts

normal          3526
magn/spot        572
wire loc          57
stereotactic      25
other              9
Name: view, dtype: int64

In [9]:
pd.crosstab(dfpred[~dfpred.view.isnull()].view.map(lambda x: colmap[x]), dfpred.set)[["train", "test", "val"]]

set,train,test,val
view,,,
magn/spot,380,96,96
normal,2310,612,604
other,4,3,2
stereotactic,17,4,4
wire loc,37,11,9


## Read image-based predictions (general)

In [14]:
tag = "e5ce2d69b035975cb5336cec0da9a32a"
epoch = 55
infile = f"{tabledir}/spotmag_predictions/predictions_images_4189-epoch{epoch}-{tag}.csv"
# infile = f"{tabledir}/spotmag_predictions/df_dcm_reports_birads_path_indic_dens_birad_wi_year_noreport_nodupl-spotmag_img_prediction-{tag}.csv"
print(infile)
dfpred_img = pd.read_csv(infile, index_col=0)
dfpred_img = dfpred_img[['score_image', 'score_image_max']]
dfpred_img = dfpred_img.groupby(level=0).mean()
dfpred_img.columns

../tables//spotmag_predictions/predictions_images_4189-epoch55-e5ce2d69b035975cb5336cec0da9a32a.csv


Index(['score_image', 'score_image_max'], dtype='object')

## Read image-based predictions (wire localization)

In [15]:
infile = f"{tabledir}/spotmag_predictions/predictions_wire_combined_e8e71fc090141d7c6fb334359152d295.csv"

dfpred_imgwire = pd.read_csv(infile, index_col=0)
dfpred_imgwire["score_wire_max"] = 1-dfpred_imgwire[["scores_0_or","scores_0_fl"]].min(1)
dfpred_imgwire = dfpred_imgwire.drop(["scores_0_or","scores_0_fl", "label"], axis=1)
dfpred_imgwire.columns = [cc.replace("scores", "score_wire") for cc in dfpred_imgwire.columns]

In [16]:
if 'score_image' not in dfpred.columns:
    dfpred = pd.concat([dfpred, dfpred_img], axis=1)
    dfpred.index.name = 'id'
    del dfpred_img
    
if 'score_glmnet' not in dfpred.columns:
    dfpred = pd.concat([dfpred, dfpred_glmnet], axis=1)
    dfpred.index.name = 'id'
    del dfpred_glmnet
    
if 'score_wire' not in dfpred.columns:
    dfpred = pd.concat([dfpred, dfpred_imgwire], axis=1)
    dfpred.index.name = 'id'
    del dfpred_imgwire

dfpred.shape

(772367, 13)

In [17]:
if 'label' not in dfpred.columns:
    dfpred = pd.concat([dfpred, dflab], axis=1)
if 'digital' not in dfpred.columns:
    dfpred = pd.concat([dfpred, df_bt], axis=1)

In [18]:
pd.crosstab(dfpred["score_wire"].isnull(), dfpred["score_image"].isnull())

score_image,False,True
score_wire,,
False,3584,0
True,605,768234


In [19]:
dfpred.rename(columns={"score_xgbt":"score_gbmt"}, inplace=True)

## Add ensembled (max, avg) scores

In [24]:
dfpred['score_wire'] = dfpred['score_wire'].fillna(0)
dfpred['score_wire_max'] = dfpred['score_wire_max'].fillna(0)
dfpred['score_image+glmnet'] = (dfpred['score_image'] + dfpred['score_glmnet'])/2
dfpred['score_image+gbmt'] = (dfpred['score_image'] + dfpred['score_gbmt'])/2

dfpred['score_max(image;gbmt)'] = dfpred[['score_image','score_gbmt']].max(1)

dfpred['score_image*glmnet'] = np.sqrt(dfpred['score_image'] * dfpred['score_glmnet'])
dfpred['score_image*gbmt'] = np.sqrt(dfpred['score_image'] * dfpred['score_gbmt'])
dfpred['score_max_image_wire'] = np.nanmax(dfpred[['score_image','score_wire']].values, axis=1)
dfpred['score_max_image_wire_max'] = np.nanmax(dfpred[['score_image','score_wire_max']].values, axis=1)
# dfpred['score_wire'].isnull()
dfpred['score_max_image_wire+gbmt'] =(dfpred['score_max_image_wire'] + dfpred['score_gbmt'])/2

dfpred['score_max_image_wire_max+gbmt'] =(dfpred['score_max_image_wire_max'] + dfpred['score_gbmt'])/2

dfpred['score_max(image;wire_max;gbmt)'] = dfpred[['score_wire_max','score_gbmt', 'score_image']].max(1)

dfpred['score_max_wire_image+gbmt'] = np.nanmax(dfpred[['score_image+gbmt','score_wire']].values, axis=1)

dfpred['score_max_wire_max_image+gbmt'] = np.nanmax(dfpred[['score_image+gbmt','score_wire_max']].values, axis=1)

In [25]:
dfpred.rename(columns={"ViewModifierCodeMeaning":"ViewModifier"}, inplace=True)
dfpred.index.name = 'id'

## Save the combined table

In [23]:
len(dfpred)

772423

In [26]:
dfpred.to_csv(f'{tabledir}/all_predictions_with_images-{tag}.tab', sep='\t')